In [ ]:
clc
clear
close all
tic
%%
Pathname = ['D:\dataset\健康人数据' filesep 'data_64'] ;
Listing = dir(Pathname) ;
filename = {Listing(3:end).name}' ;
% 获取文件名并使用natsort进行排序
filename = natsort(filename);
R_ITPC_37_1=[];
R_ITPC_37_2=[];
R_ITPC_37_3=[];
for isSub =1:20
    isSub
    Pathname1 = [Pathname filesep filename{isSub}] ;
    Listing1 = dir(Pathname1) ;
    filename1 = {Listing1(3:end).name}' ;
    for days=1:3
           Data=load( [Pathname1 filesep filename1{days} filesep 'ITPC_2.mat'] );
       ITPC(isSub,days,:,:)=Data.itpc_81;
    end  
end
times    = linspace(-100,600,1400);
%frequencies_81 = logspace(log10(20), log10(100), 150);%1-50Hz
frequencies_81 = linspace(20, 100, 150);
%% 整体ICC计算
%ITPC_37=squeeze(mean(ITPC_37(:,:,:,:),1));
 R_ITPC_37_1=squeeze(mean(ITPC(:,1,:,:),1));
  R_ITPC_37_2=squeeze(mean(ITPC(:,2,:,:),1));
 R_ITPC_37_3=squeeze(mean(ITPC(:,3,:,:),1));
itpc_data=ITPC;
% 计算 ICC
icc_values = nan(size(itpc_data, 3), size(itpc_data, 4)); % 初始化 ICC 结果矩阵,频率*时间

for freq_idx = 1:size(itpc_data, 3)
    for time_idx = 1:size(itpc_data, 4)
        %data_point = squeeze(itpc_data(:,freq_idx, time_idx)); % 提取当前频率-时间点的数据
        icc_values(freq_idx, time_idx) =ICC([squeeze(itpc_data(:,1,freq_idx,time_idx)),squeeze(itpc_data(:,2,freq_idx,time_idx)),squeeze(itpc_data(:,3,freq_idx,time_idx))], 'A-1');; % 使用 ICC 函数计算 ICC
    end
end
% 绘制 ICC 结果
figure(1)
% itpc=squeeze(mean(itpc,3));
contourf(times,frequencies_81,icc_values,150,'linecolor','none')
set(gca,'clim',[0 1],'xlim',[-100 600])
xlabel('Time (ms)'), ylabel('Frequencies (Hz)')
yline(37, '--', 'LineWidth', 1.5)
% 添加 colorbar 并命名
cb = colorbar;
cb.Label.String = 'ICC Values';  % 设置 colorbar 的标签
%% 
alpha=0.05
mean_icc=mean(icc_values(:));
std_icc=std(icc_values(:));
z_score = (icc_values - mean_icc) / std_icc;
p_values = 1 - normcdf(abs(z_score));%单尾
% 第二步：可视化显著区域
% 假设您想要可视化p < alpha的区域

% 创建一个二进制掩码，其中显著区域标记为1
significant_mask = p_values < alpha;

% 图块显示
figure(2);
% 绘制ITPC_1的等高线图
contourf(times, frequencies_81, icc_values, 150, 'linecolor', 'none')
set(gca, 'clim', [0 1], 'xlim', [-100 600])
hold on;

% 将显著区域的颜色全部变为红色
colormap(gca, [0.5 0.5 1; 1 0.5 0.5]); % 使用灰色和红色的颜色映射
imagesc(Data.times, frequencies_37, significant_mask, 'AlphaData', significant_mask)
xlabel('时间 (ms)'), ylabel('频率 (Hz)')
yline(37, '--', 'LineWidth', 1.5)
%% 37hZ处的ITPC值的ICC 37hz对应83
figure(3)
% 提取第一行数据
row_data = icc_values(83, :);

% 绘制箱线图
boxplot(row_data);
title('ICC 37');
xlabel('Variables');
ylabel('Values');

figure(4)
row_data = squeeze(icc_values(83, :));
%scatter(time, row_data); % 绘制第一行数据的散点图
plot(time,row_data,'r','linewidth', 2);
% 设置坐标轴标签
% xlabel('N1与P2的峰值和潜伏期','fontsize',15);
ylabel('ICC','fontsize',15);
% 纵坐标为0.39的虚线
yline(0.39, '--', 'LineWidth', 1.5);
% 纵坐标为0.59的虚线
yline(0.59, '--', 'LineWidth', 1.5);
% 纵坐标为0.75的虚线
yline(0.75, '--', 'LineWidth', 1.5)
ylim([0, 1]);
title('37 Hz处的ITPC');
xlabel('Time');
ylabel('ICC');
set(gca,'fontsize',25)